In [114]:
import pandas as pd
import numpy as np
import json
import allel

## Truth data is gathered from the LTEE Stanford experiment. Data is obtained from Good, B., McDonald, M., Barrick, J. et al. The dynamics of molecular evolution over 60,000 generations. Nature 551, 45–50 (2017). https://doi.org/10.1038/nature24287
and
Tenaillon, O., Barrick, J., Ribeck, N. et al. Tempo and mode of genome evolution in a 50,000-generation experiment. Nature 536, 165–170 (2016). https://doi.org/10.1038/nature18959

In [105]:
#df = pd.read_csv("~/Downloads/p1_well_mixed_state_timecourse.txt")

In [106]:
df.columns

Index(['0', ' 1000', ' 1500', ' 2000', ' 2500', ' 3000', ' 4000', ' 4500',
       ' 5000', ' 5500',
       ...
       ' 55500', ' 56000', ' 56500', ' 57000', ' 57500', ' 58000', ' 58500',
       ' 59000', ' 59500', ' 60000'],
      dtype='object', length=117)

In [107]:
columns_to_drop = set()
for col in df.columns:
    if int(col) > 50000:
        columns_to_drop.add(col)

In [108]:
df.drop(columns=columns_to_drop, inplace=True)

In [109]:
np_df = df.T.to_numpy()

In [110]:
allele_freq_var = np.var(np_df/np_df.max())

In [111]:
"""
with open("/Users/berk/Projects/jlees/data/truth_data/p1/p1_allele_freq_var.json", "w") as fh:
    json.dump(
        {"allele_freq_var" : allele_freq_var}, fh
    )
"""

In [113]:
del df
del np_df

In [126]:
## Mutation rates are calculated from the spectrum_counts.csv file from the Good, B., McDonald, M., Barrick, J. et al. The dynamics of molecular evolution over 60,000 generations. Nature 551, 45–50 (2017). https://doi.org/10.1038/nature24287
## Note that only SNP mutations are taken into account
##mutation_rate = 92/50000
with open("/Users/berk/Projects/jlees/data/truth_data/m6/m6_mutation_rate.json", "w") as fh:
    json.dump(
        {"mutation_rate" : mutation_rate}, fh
    )

## Constructing Haplotype array

In [127]:
df_mut_array = pd.read_csv("~/Downloads/LTEE-Ecoli-data 2023-08-08 03_55_51.csv")

In [129]:
df_cleaned = df_mut_array.drop(columns=["Unnamed: 0", "time", "mutation_category", "mutator_status", 'html_mutation_annotation', 'treatment', 'clone', 'type', 'snp_type', 'start_position', 'gene_list', 'gene_name', 'gene_position', 'gene_product', 'locus_tag', 'html_gene_name', 'html_gene_product', 'html_position'])

In [131]:
df_cleaned.groupby('population').count()

,strain,end_position,html_mutation
population,,,
Ara+1,336,336,336
Ara+2,311,311,311
Ara+3,10799,10799,10799
Ara+4,328,328,328
Ara+5,359,359,359
Ara+6,18859,18859,18859
Ara-1,4712,4712,4712
Ara-2,7549,7549,7549
Ara-3,1800,1800,1800


In [380]:
ara = df_cleaned[df_cleaned['population'] == "Ara-1"]

In [381]:
max_mut_pos = ara['end_position'].max()

In [382]:
n_strains = ara['strain'].nunique()

In [383]:
hap_array = np.zeros([n_strains, max_mut_pos+1], dtype=np.int8)

In [384]:
all_positions = set()
for idx, pos in enumerate(ara.groupby('strain')['end_position']):
    for p in list(pos[1]):
        all_positions.add(p)
        hap_array[idx,p] += 1

In [385]:
haplotype_array = allel.HaplotypeArray(hap_array)

In [386]:
poses = list(sorted(all_positions))
allele_counts_array = haplotype_array.count_alleles()

In [387]:
pi = allel.sequence_diversity(pos=poses, ac=allele_counts_array)

In [388]:
theta = allel.watterson_theta(poses, allele_counts_array)

In [389]:
tajimas_d = allel.tajima_d(allele_counts_array)

In [390]:
garuda_h = allel.garud_h(haplotype_array)

In [391]:
hap_div = allel.haplotype_diversity(haplotype_array)

In [392]:
seq_stats = {   'pi': pi,
    'theta_w': theta,
    'tajimas_d': tajimas_d,
    'h1':garuda_h[0],
    'h2_h1':garuda_h[3],
    'haplotype_diversity':hap_div
}

In [393]:
with open('/Users/berk/Projects/jlees/data/truth_data/m1/seq_stats.json', 'w') as fhand:
    json.dump(
        seq_stats, fhand
    )

In [394]:
del hap_array
del haplotype_array
del seq_stats

## Unite all the files to a single csv

In [411]:
nn_model_indices = {
    'max_H' : 0,
    'min_H' : 1,
    'a_BL_mean' : 2,
    'a_BL_median' : 3,
    'e_BL_mean' : 4,
    'e_BL_median' : 5,
    'e_BL_var' : 6,
    'i_BL_mean_1': 7,
    'i_BL_median_1': 8,
    'ie_BL_median_1' : 9,
    'i_BL_mean_2' : 10,
    'i_BL_median_2' : 11,
    'i_BL_var_2' : 12,
    'ie_BL_mean_2' : 13,
    'ie_BL_median_2' : 14,
    'ie_BL_var_2' : 15,
    'i_BL_mean_3' : 16,
    'i_BL_median_3' : 17,
    'i_BL_var_3' : 18,
    'ie_BL_median_3' : 19,
    'colless' : 20,
    'WD_ratio' : 21,
    'delta_w' : 22,
    'max_ladder' : 23,
    'IL_nodes' : 24,
    'staircaseness_1' : 25,
    'staircaseness_2' : 26,
    'pi' : 27,
    'theta_w' : 28,
    'tajimas_d' : 29,
    'h1' : 30,
    'h2_h1' : 31,
    'haplotype_diversity' : 32,
    'allele_freq_var' : 33,
    'n_individuals' : 34,
    'mutation_rate' : 35
}

In [412]:
import os
ov = []
_path = '/Users/berk/Projects/jlees/data/truth_data'
for folder in os.listdir(_path):
    f_dict= {}
    for file in os.listdir(os.path.join(_path, folder)):
        file_path = os.path.join(_path, folder, file)
        with open(file_path, 'r') as fh:
            f_dict.update(json.load(fh))
        f_dict.update({'n_individuals' : 5000000})
    ov.append(f_dict)


In [419]:
empty_array = np.zeros([12, len(nn_model_indices)], dtype=np.float32)

In [420]:
out_df = pd.DataFrame(empty_array, columns=nn_model_indices, dtype=np.float32)

In [421]:
for idx, _dict in enumerate(ov):
    for key in nn_model_indices.keys():
        out_df[key][idx] = _dict[key]

In [422]:
###Save df
out_df.to_csv(os.path.join(_path, 'truth_data_stats.csv'), index=False)